### Summary of RESQML Entity Relationships

In RESQML, a WellboreTrajectoryRepresentation is typically linked to a WellboreInterpretation, and seems to be the _main_ entity as it defines the physical mapping of the borehole itself. In RESQML data we can't necessarily rely on WellboreFeature existing, so when decoding it isn't sufficient to always look for WellboreFeature --> WellboreInterpretation --> WellboreTrajectory relationships. Instead go straight for the Trajectories.

More on the relationships (where they exist):

#### WellboreFeature
Repo: resqpy/organize/wellbore_feature.py
* The Wellbore Feature is linked to a WellboreInterpretation via uuid.
* It is an 'organisational' object, which has its own uuid so it can be linked to by other objects, and may contain Well title/name, originator info (optional string), and extra metadata.

#### WellboreInterpretation
Repo: resqpy/organize/wellbore_interpretation.py
* The WellboreInterpretation is the physical instance of a WellboreFeature.
* A WellboreTrajectoryRepresentation Trajectory is directly linked to a WellboreInterpretation, which represents the interpretative understanding of the well path associated with a WellboreFeature (such as a specific well).

#### WellboreTrajectoryRepresentation:
Repo: resqpy/well/_trajectory.py
* Defines the physical path of the wellbore, including measured depths (MDs), inclinations, and azimuths.
* Is linked to a WellboreInterpretation, representing an interpretative understanding of a specific well path.
* Provides the foundational axis along which intervals are defined.

#### WellboreFrameRepresentation:
Repo: resqpy/well/_wellbore_frame.py
* A WellboreFrameRepresentation is a type of depth-indexed frame that provides additional structure along the trajectory, often used to capture well log data and downhole interval properties.
* Acts as a "depth-indexed frame" along the trajectory path, where depth (usually MD) values define interval nodes.
* Downhole intervals are defined between pairs of depth nodes (e.g., MD 1000 to MD 1200).
* The WellboreFrameRepresentation references the WellboreTrajectoryRepresentation as its underlying path but does not link back to the WellboreInterpretation directly.

#### Downhole Interval Data (Properties):
* Stored as log data or interval properties within a WellboreFrameRepresentation.
* Each interval can hold various properties like porosity, resistivity, or lithology, recorded as ContinuousProperties or CategoricalProperties.
* These properties are often stored in WellLogCollection objects associated with the frame and are indexed along the trajectory's measured depths.

### Unpack Approach for Properties associated with WellboreFrames
1. Read all WellboreTrajectoryRepresentations in the model
2. For each Trajectory iterate over all WellboreFrameRepresentations linked to it
3. For each WellboreFrameRepresentation find all Properties linked to it

### Unpack Approach for Properties associated with WellLogCollections
1. Read all WellboreTrajectoryRepresentations in the model
2. For each Trajectory iterate over all WellboreFrameRepresentations linked to it
3. For each WellboreFrameRepresentation find all WellLogCollections
4. For each WellLogCollection unpack the WellLogs (properties)


In [ ]:
from pprint import pprint

import numpy as np
import pandas as pd
import resqpy.crs as rqcrs
import resqpy.model as rq
import resqpy.property as rqp
import resqpy.property.property_common as rqp_c
import resqpy.well as rqw

from evo.data_converters.common import EvoWorkspaceMetadata, create_evo_object_service_and_data_client
from evo.data_converters.resqml.importer.resqml_to_evo import _convert_downhole_intervals

#### Acquire a data client

In [ ]:
meta_data = EvoWorkspaceMetadata(
    org_id="",
    workspace_id="",
)
_, data_client = create_evo_object_service_and_data_client(meta_data)

#### Create a model containing a test Well with Trajectory and associated downhole interval properties

In [ ]:
# RESET: Create a new empty RESQML dataset (will overwrite existing)
epc_file = "data/test-wellbore-data.epc"
m = rq.new_model(epc_file)
m.store_epc()

# Load the RESQML model
epc_file = "data/test-wellbore-data.epc"
model = rq.Model(epc_file)

# Create a CRS
crs = rqcrs.Crs(parent_model=model, epsg_code="4326", title="EPSG:4326")
crs.create_xml()

# Set our model to use that CRS. Well objects created in this
# model can then use it as default (or set it independently)
model.crs_uuid = crs.uuid

# Add a Trajectory definition for the well.
trajectory_df = pd.DataFrame(
    (
        (145.899994, 15051.129883, 8561.570312, 90.999992),
        (218.500000, 15051.081055, 8561.413086, 163.599762),
        (259.000000, 15050.992188, 8561.308594, 204.099487),
        (299.500000, 15050.823242, 8561.232422, 244.599030),
        (340.000000, 15050.803711, 8561.299805, 285.098663),
        (394.299988, 15050.877930, 8561.629883, 339.397522),
        (420.399994, 15050.846680, 8561.786133, 365.497040),
        (460.600006, 15050.901367, 8562.007812, 405.696381),
        (500.799988, 15051.045898, 8562.207031, 445.895599),
        (540.900024, 15052.113281, 8562.105469, 485.977386),
        (580.400024, 15054.400391, 8561.649414, 525.407959),
        (620.799988, 15057.659180, 8560.903320, 565.667603),
        (661.500000, 15062.120117, 8560.461914, 606.117188),
        (701.799988, 15067.651367, 8560.851562, 646.032104),
        (742.200012, 15074.749023, 8561.744141, 685.788574),
        (782.599976, 15082.649414, 8563.025391, 725.386841),
        (823.000000, 15090.331055, 8564.900391, 765.004150),
        (863.299988, 15097.926758, 8567.539062, 804.492676),
        (902.599976, 15104.895508, 8569.920898, 843.094116),
        (968.020020, 15115.603516, 8572.855469, 907.564209),
        (1008.500000, 15122.037109, 8574.555664, 947.493042),
        (1048.859985, 15128.499023, 8576.191406, 987.298340),
        (1083.000000, 15134.030273, 8577.557617, 1020.959351),
    ),
    columns=("MD", "X", "Y", "Z"),
)


# Establish an MdDatum object for the well and set it vertically
# above the first trajectory control point.
datum_xyz = trajectory_df["X"][0], trajectory_df["Y"][0], trajectory_df["Z"][0] - trajectory_df["MD"][0]
md_datum = rqw.MdDatum(
    parent_model=model,
    crs_uuid=model.crs_uuid,  # handy if all your objects use the same crs
    location=datum_xyz,
    md_reference="ground level",
    title="spud datum",
)
md_datum.create_xml()

# Create the well Trajectory
trajectory = rqw.Trajectory(
    well_name="Test Well #1",
    parent_model=model,
    md_datum=md_datum,
    data_frame=trajectory_df,
    length_uom="m",  # this is the md_uom
)

# Create the WellboreFeature and WellboreInterpretation objects
trajectory.create_feature_and_interpretation()

# Add the trajectory (and related objects) permanently to our model
trajectory.write_hdf5()
trajectory.create_xml()

# A WellboreFrame is used as an entity to associate with downhole intervals
# It follows the well trajectory but can define its own measured depths for
# each set of downhole interval data. In this example, we are setting 5
# arbitrary depths which matches the number of downhole interval properties
# we will be adding.
wellbore_frame = rqw.WellboreFrame(
    parent_model=model,
    trajectory=trajectory,
    mds=[420.8, 433.4, 505.5, 530.28, 610.18],
    title="Test wellbore measurements #1",
)

# Creates WellboreFeature, and WellboreInterpretation objects
# related to the WellboreFrame and its Trajectory
wellbore_frame.write_hdf5()
wellbore_frame.create_xml()

# Create the downhole interval data.
# This is a mix of Categorical and Continuous properties.
# Lookup table for some categorical lithology data
lithology_lookup = rqp.StringLookup(
    model, title="lithology", int_to_str_dict={0: "Sandstone", 1: "Shale", 2: "Limestone"}
)
lithology_lookup.create_xml()

# Create the intervals datasets
interval_data = {
    "porosity": [0.15, 0.18, 0.22, 0.20, 0.17],
    "temperature": [40, 60, 80, 90, 100],
    "formation_pressure": [1500, 2000, 2500, 3000, 3500],
    "lithology": [0, 1, 1, 2, 0],
}


# Function to add properties to the model. The default property type is continuous.
# For categorical properties set discrete=True, and string_lookup_uuid to a valid
# string lookup table UUID.
def add_wellbore_property(frame, values, title, uom=None, is_discrete=False, string_lookup_uuid=None):
    # Convert values to numpy array if not already
    prop_array = np.array(values)

    # Create property object
    rqp.Property.from_array(
        parent_model=model,
        cached_array=prop_array,
        source_info="interval data",
        keyword=title,
        property_kind=title,
        uom=uom,
        support_uuid=frame.uuid,
        indexable_element="nodes",
        discrete=is_discrete,
        string_lookup_uuid=string_lookup_uuid,
    ).create_xml()


# Adding interval properties to the frame
add_wellbore_property(frame=wellbore_frame, values=interval_data["porosity"], title="porosity", uom="fraction")
add_wellbore_property(frame=wellbore_frame, values=interval_data["temperature"], title="temperature", uom="F")
add_wellbore_property(
    frame=wellbore_frame, values=interval_data["formation_pressure"], title="formation_pressure", uom="mPA"
)
add_wellbore_property(
    frame=wellbore_frame,
    values=interval_data["lithology"],
    title="lithology",
    string_lookup_uuid=lithology_lookup.uuid,
    is_discrete=True,
)


# Now add properties as WellLogs to this frame
def add_welllog(welllog_collection, values, title, uom=None, is_discrete=False, string_lookup_uuid=None):
    prop_array = np.array(values)

    property_kind, facet_type, facet = rqp_c.infer_property_kind(title, uom)

    welllog_collection.add_cached_array_to_imported_list(
        cached_array=prop_array,
        source_info="interval data",
        keyword=title,
        property_kind=property_kind,
        uom=uom,
        indexable_element="nodes",
        discrete=is_discrete,
        string_lookup_uuid=string_lookup_uuid,
    )


welllog_collection = rqp.WellLogCollection(wellbore_frame)
add_welllog(welllog_collection=welllog_collection, values=interval_data["porosity"], title="porosity2", uom="fraction")
add_welllog(
    welllog_collection=welllog_collection, values=interval_data["temperature"], title="temperature2", uom="degF"
)
add_welllog(
    welllog_collection=welllog_collection,
    values=interval_data["formation_pressure"],
    title="formation_pressure2",
    uom="psi",
)
add_welllog(
    welllog_collection=welllog_collection,
    values=interval_data["lithology"],
    title="lithology2",
    uom="Euc",
    string_lookup_uuid=lithology_lookup.uuid,
    is_discrete=True,
)

welllog_collection.write_hdf5_for_imported_list()
welllog_collection.create_xml_for_imported_list_and_add_parts_to_model()

# Finalize the model and save it
model.store_epc()
model.h5_release()

#### Run the converter on the model created above

In [ ]:
# Import from the test model we just created
dhis = _convert_downhole_intervals(model, data_client)
pprint(dhis)

#### Optional: Run the converter on the Volve Wells model
##### Download the data and place into samples/data-converters/python/convert-resqml/data
Note that this import will take a while due to datset size

In [ ]:
# Import from the Volve Wells RESQML model
epc_file = "data/Volve_Demo_Wells_Depth.epc"
model = rq.Model(epc_file)

dhis = _convert_downhole_intervals(model, data_client)
pprint(dhis)